In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from collections import Counter
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score,f1_score, auc, precision_recall_curve, accuracy_score, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks

# Load Data

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Datasets/VirusShare_Opcodes/csvFiles/DATASET_SHUFFLED_VirusShare_proportions_and_targets.csv')\
df = pd.read_csv('/content/drive/MyDrive/Datasets/VirusShare_Opcodes/csvFiles/MalwareFamily_DATASET_FINAL_Increased_Imbalance.csv')
# Delete rows where 'name' is equal to 'Unknown_Family'
df = df[(df['name'] != 'Unknown_Family')]
df['name'].value_counts()

name
Vobfus         4204
Zbot           2353
Diplugem       2269
Obfuscator     2102
Vundo          1877
VBInject       1688
Delf           1679
Beebone        1629
Winwebsec      1625
Enterak.A      1530
OnLineGames     137
Startpage       131
Allaple.A       129
Injector        116
Systex.A        110
Expiro.BK       110
FakeRean        109
Small           105
Toga!rfn         25
Lamechi.B        25
Name: count, dtype: int64

In [ ]:
df['Family Target'].value_counts()

Family Target
1     4204
2     2353
3     2269
4     2102
5     1877
6     1688
7     1679
8     1629
9     1625
10    1530
11     137
12     131
13     129
14     116
15     110
16     110
17     109
18     105
19      25
20      25
Name: count, dtype: int64

In [ ]:
df['Family Target'].value_counts()/len(df)

Family Target
1     0.191500
2     0.107184
3     0.103357
4     0.095750
5     0.085501
6     0.076892
7     0.076482
8     0.074204
9     0.074022
10    0.069694
11    0.006241
12    0.005967
13    0.005876
14    0.005284
15    0.005011
16    0.005011
17    0.004965
18    0.004783
19    0.001139
20    0.001139
Name: count, dtype: float64

In [ ]:
df.head()

,file_name,mov,add,push,pop,inc,call,xor,dec,cmp,...,insb,jae,outsb,popa,jo,ja,name,Category,Category Target,Family Target
0,VirusShare_61414397fe0108e7f3d729d4372850cd,0.176383,0.102180,0.170927,0.014749,0.027229,0.109421,0.007918,0.005528,0.006880,...,0.000797,0.007676,0.001593,0.006590,0.002607,0.000628,Vobfus,Worm,2,1
1,VirusShare_f6409ecf531a795368a0172ee19a1e09,0.035199,0.342921,0.040362,0.027478,0.039454,0.011811,0.012601,0.014605,0.016986,...,0.017364,0.003466,0.005140,0.002688,0.005788,0.001049,Vobfus,Worm,2,1
2,VirusShare_3466fc13ccaffd1202657fcfd72c8692,0.045042,0.305280,0.047044,0.022496,0.033230,0.005754,0.020863,0.017465,0.023421,...,0.013006,0.003432,0.003315,0.002557,0.006192,0.001380,Vobfus,Worm,2,1
3,VirusShare_fca52c87dadb1bf3ea7bf2d526edd12c,0.103277,0.102014,0.214684,0.015998,0.030507,0.137185,0.003077,0.004825,0.001814,...,0.001263,0.005020,0.001781,0.008517,0.003271,0.000745,Vobfus,Worm,2,1
4,VirusShare_93b8eb2c79d46e92ec816844a41fdbaa,0.182714,0.114227,0.179367,0.014457,0.027968,0.102761,0.008104,0.005888,0.010707,...,0.002045,0.004555,0.002092,0.006074,0.003719,0.000527,Vobfus,Worm,2,1


# Features/Targets

In [ ]:
# Prepare the features
features = df.drop(columns=["file_name","name", "Category","Category Target", "Family Target"]).astype(float)

# Prepare the target
targets = df["Family Target"].astype(int)

In [ ]:
features.shape

(21953, 40)

In [ ]:
targets.shape

(21953,)

# Split Train/Test Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    targets,
    stratify=targets,
    test_size=0.2,
    random_state=0)


X_train.shape, X_test.shape

((17562, 40), (4391, 40))

# Metric Functions

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score

def calculate_metrics(y_true, y_pred):
    # Calculate metrics
    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    # Calculate AUC-ROC & Precision-Recall for each class
    for i in range(1, (df['name'].nunique() + 1)):
        print(f"AUC-ROC for class {i}: {roc_auc_score(y_true == i, y_pred == i)}")
        print(f"PR AUC for class {i}: {average_precision_score(y_true == i, y_pred == i)}")

    # Calculate Accuracy
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))

In [ ]:
def calculate_MacroAvg_metrics(description, y_pred, y_test):

    # Calculate each metric
    metrics = classification_report(y_test, y_pred, output_dict=True)
    accuracy = accuracy_score(y_test, y_pred)

    # Store the metrics in a DataFrame
    metrics_df = pd.DataFrame({
        'Model': description,
        'Precision': metrics['macro avg']['precision'],
        'Recall': metrics['macro avg']['recall'],
        'F1-Score': metrics['macro avg']['f1-score'],
        'Accuracy': accuracy
    },index=[0])

    return metrics_df
final_result = []

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import pandas as pd

def generate_multilabel_confusion_df(y_true, y_pred):
  labels = range(1, (df['name'].nunique() + 1))
  cms = multilabel_confusion_matrix(y_true, y_pred, labels=labels)
  confusion_matrices = {}
  for cm, label in zip(cms, labels):
      confusion_df = pd.DataFrame({
          'Predicted -': [cm[0][0], cm[1][0]],
          'Predicted +': [cm[0][1], cm[1][1]]
      }, index=['Actual -', 'Actual +'])
      confusion_matrices[label] = confusion_df
  return confusion_matrices

# def plot_confusion_matrix(y_true, y_pred):
#   labels = range(df['name'].nunique())
#   cm = confusion_matrix(y_true, y_pred, labels=labels)
#   plt.figure(figsize=(10,7))
#   sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
#   plt.xlabel('Predicted')
#   plt.ylabel('True')
#   plt.show()


# Models

## KNN model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
# Predict the labels for the test set
y_pred_knn = knn.predict(X_test)
metrics_knn = calculate_MacroAvg_metrics("KNN (Unbalanced)",y_pred_knn, y_test)
final_result.append(metrics_knn)

print("-----------------------------KNN (Unbalanced)-----------------------------\n")
calculate_metrics(y_test, y_pred_knn)

-----------------------------KNN (Unbalanced)-----------------------------

Classification Report:
              precision    recall  f1-score   support

           1       0.96      0.98      0.97       841
           2       0.78      0.85      0.81       471
           3       1.00      1.00      1.00       454
           4       0.78      0.74      0.76       420
           5       0.90      0.89      0.89       375
           6       0.75      0.74      0.74       338
           7       0.99      0.96      0.97       336
           8       0.94      0.97      0.96       326
           9       0.82      0.80      0.81       325
          10       0.95      0.97      0.96       306
          11       0.74      0.85      0.79        27
          12       1.00      0.73      0.84        26
          13       0.96      1.00      0.98        26
          14       0.64      0.30      0.41        23
          15       0.88      1.00      0.94        22
          16       0.80      0.36   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

### Confusion_Matrix_KNN

In [ ]:
confusion_knn = generate_multilabel_confusion_df(y_test, y_pred_knn)
for label, frame in confusion_knn.items():
    print(f'Confusion Matrix for {label}:')
    print(frame)
    print()

Confusion Matrix for 1:
          Predicted -  Predicted +
Actual -         3512           38
Actual +           14          827

Confusion Matrix for 2:
          Predicted -  Predicted +
Actual -         3808          112
Actual +           72          399

Confusion Matrix for 3:
          Predicted -  Predicted +
Actual -         3937            0
Actual +            1          453

Confusion Matrix for 4:
          Predicted -  Predicted +
Actual -         3885           86
Actual +          110          310

Confusion Matrix for 5:
          Predicted -  Predicted +
Actual -         3980           36
Actual +           43          332

Confusion Matrix for 6:
          Predicted -  Predicted +
Actual -         3969           84
Actual +           88          250

Confusion Matrix for 7:
          Predicted -  Predicted +
Actual -         4051            4
Actual +           14          322

Confusion Matrix for 8:
          Predicted -  Predicted +
Actual -         4046          

## RF model

In [ ]:
# Train the model
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
# Predict the labels for the test set
y_pred_rf = rf.predict(X_test)
metrics_rf = calculate_MacroAvg_metrics("Random Forest (Unbalanced)",y_pred_rf,  y_test)
final_result.append(metrics_rf)
print("-----------------------------Random Forest (Unbalanced)-----------------------------\n")
calculate_metrics(y_test, y_pred_rf)
#plot_confusion_matrix(y_test, y_pred_rf)

-----------------------------Random Forest (Unbalanced)-----------------------------

Classification Report:
              precision    recall  f1-score   support

           1       0.99      0.99      0.99       841
           2       0.84      0.87      0.86       471
           3       1.00      1.00      1.00       454
           4       0.79      0.84      0.81       420
           5       0.96      0.90      0.93       375
           6       0.79      0.88      0.83       338
           7       0.99      0.98      0.98       336
           8       0.99      0.99      0.99       326
           9       0.92      0.88      0.90       325
          10       1.00      0.98      0.99       306
          11       1.00      0.89      0.94        27
          12       1.00      0.81      0.89        26
          13       1.00      1.00      1.00        26
          14       0.88      0.61      0.72        23
          15       1.00      1.00      1.00        22
          16       1.00   

### Confusion_Matrix_RF

In [ ]:
confusion_rf = generate_multilabel_confusion_df(y_test, y_pred_rf)
for label, frame in confusion_rf.items():
    print(f'Confusion Matrix for {label}:')
    print(frame)
    print()

Confusion Matrix for 1:
          Predicted -  Predicted +
Actual -         3542            8
Actual +            8          833

Confusion Matrix for 2:
          Predicted -  Predicted +
Actual -         3844           76
Actual +           59          412

Confusion Matrix for 3:
          Predicted -  Predicted +
Actual -         3937            0
Actual +            1          453

Confusion Matrix for 4:
          Predicted -  Predicted +
Actual -         3878           93
Actual +           68          352

Confusion Matrix for 5:
          Predicted -  Predicted +
Actual -         4003           13
Actual +           37          338

Confusion Matrix for 6:
          Predicted -  Predicted +
Actual -         3974           79
Actual +           40          298

Confusion Matrix for 7:
          Predicted -  Predicted +
Actual -         4050            5
Actual +            7          329

Confusion Matrix for 8:
          Predicted -  Predicted +
Actual -         4061          

## SVM model

In [ ]:
# Train the model
svm = SVC(kernel='rbf', random_state=0,class_weight= None, gamma= 'scale')
svm.fit(X_train, y_train)

SVC(random_state=0)

In [ ]:
# Predict the labels for the test set
y_pred_svm = svm.predict(X_test)
metrics_svm = calculate_MacroAvg_metrics("SVM (Unbalanced)",y_pred_svm, y_test)
final_result.append(metrics_svm)
print("-----------------------------SVM (Unbalanced)-----------------------------\n")
calculate_metrics(y_test, y_pred_svm)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

-----------------------------SVM (Unbalanced)-----------------------------

Classification Report:
              precision    recall  f1-score   support

           1       0.91      0.97      0.94       841
           2       0.53      0.53      0.53       471
           3       1.00      0.99      1.00       454
           4       0.41      0.69      0.52       420
           5       0.65      0.67      0.66       375
           6       0.76      0.44      0.56       338
           7       0.97      0.96      0.97       336
           8       0.88      0.82      0.85       326
           9       0.81      0.67      0.73       325
          10       0.92      0.96      0.94       306
          11       0.00      0.00      0.00        27
          12       0.86      0.73      0.79        26
          13       0.92      0.88      0.90        26
          14       1.00      0.04      0.08        23
          15       0.00      0.00      0.00        22
          16       0.00      0.00   

### Confusion_Matrix_SVM

In [ ]:
confusion_svm = generate_multilabel_confusion_df(y_test, y_pred_svm)
for label, frame in confusion_svm.items():
    print(f'Confusion Matrix for {label}:')
    print(frame)
    print()

Confusion Matrix for 1:
          Predicted -  Predicted +
Actual -         3471           79
Actual +           27          814

Confusion Matrix for 2:
          Predicted -  Predicted +
Actual -         3701          219
Actual +          223          248

Confusion Matrix for 3:
          Predicted -  Predicted +
Actual -         3937            0
Actual +            3          451

Confusion Matrix for 4:
          Predicted -  Predicted +
Actual -         3552          419
Actual +          129          291

Confusion Matrix for 5:
          Predicted -  Predicted +
Actual -         3880          136
Actual +          125          250

Confusion Matrix for 6:
          Predicted -  Predicted +
Actual -         4006           47
Actual +          188          150

Confusion Matrix for 7:
          Predicted -  Predicted +
Actual -         4045           10
Actual +           12          324

Confusion Matrix for 8:
          Predicted -  Predicted +
Actual -         4029          

## MLP Model

In [ ]:
# Train the model
mlp = MLPClassifier(hidden_layer_sizes=(200,200, 200),early_stopping=True,random_state=0, n_iter_no_change= 5)
mlp.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(200, 200, 200),
              n_iter_no_change=5, random_state=0)

In [ ]:
# Predict the labels for the test set
y_pred_mlp = mlp.predict(X_test)
metrics_mlp = calculate_MacroAvg_metrics("MLP (Unbalanced)",y_pred_mlp, y_test)
final_result.append(metrics_mlp)
print("-----------------------------MLP (Unbalanced)-----------------------------\n")
calculate_metrics(y_test, y_pred_mlp)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

-----------------------------MLP (Unbalanced)-----------------------------

Classification Report:
              precision    recall  f1-score   support

           1       0.95      0.98      0.97       841
           2       0.69      0.63      0.66       471
           3       1.00      0.99      1.00       454
           4       0.69      0.55      0.61       420
           5       0.74      0.85      0.80       375
           6       0.58      0.83      0.68       338
           7       0.97      0.96      0.97       336
           8       0.96      0.92      0.94       326
           9       0.80      0.75      0.77       325
          10       0.94      0.96      0.95       306
          11       0.83      0.74      0.78        27
          12       1.00      0.73      0.84        26
          13       0.96      1.00      0.98        26
          14       0.67      0.09      0.15        23
          15       0.91      0.95      0.93        22
          16       0.50      0.18   

### Confusion_Matrix_MLP

In [ ]:
confusion_mlp = generate_multilabel_confusion_df(y_test, y_pred_mlp)
for label, frame in confusion_mlp.items():
    print(f'Confusion Matrix for {label}:')
    print(frame)
    print()

Confusion Matrix for 1:
          Predicted -  Predicted +
Actual -         3510           40
Actual +           17          824

Confusion Matrix for 2:
          Predicted -  Predicted +
Actual -         3786          134
Actual +          172          299

Confusion Matrix for 3:
          Predicted -  Predicted +
Actual -         3937            0
Actual +            3          451

Confusion Matrix for 4:
          Predicted -  Predicted +
Actual -         3866          105
Actual +          187          233

Confusion Matrix for 5:
          Predicted -  Predicted +
Actual -         3906          110
Actual +           55          320

Confusion Matrix for 6:
          Predicted -  Predicted +
Actual -         3851          202
Actual +           59          279

Confusion Matrix for 7:
          Predicted -  Predicted +
Actual -         4046            9
Actual +           12          324

Confusion Matrix for 8:
          Predicted -  Predicted +
Actual -         4051          

# Macro Avg Results

In [ ]:
df_concat = pd.concat(final_result, axis=0, ignore_index=True)
df_concat

,Model,Precision,Recall,F1-Score,Accuracy
0,KNN (Unbalanced),0.808752,0.767979,0.778716,0.889319
1,Random Forest (Unbalanced),0.946790,0.844784,0.877392,0.929629
2,SVM (Unbalanced),0.576424,0.492052,0.504276,0.765202
3,MLP (Unbalanced),0.798394,0.691949,0.713449,0.838306
